In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import control
import random
import networkx as nx
import pandas as pd

## Graph Setting

In [32]:
def generate_B(n, m, M):
    #print(M)
    B = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            if i == M[j]:
                B[i][j] = 1
    #print(B)
    return B

def generate_C(n, l, O):
    #print(O)
    C = np.zeros((l,n))
    for j in range(l):
        for i in range(n):
            if i == O[j]:
                C[j][i] = 1
    #print(C)
    return C

def choose_graph(graph_mode):
    g = graph_mode
    if g == 'bara':
        graph = nx.barabasi_albert_graph(n, m_ba)
    elif g == 'er':
        graph = nx.erdos_renyi_graph(n, p_er) 
    elif g == 'ws':
        graph = nx.watts_strogatz_graph(n, k_ws, p_ws)
    return graph


def weighted_matrix(n):
    sign = np.random.choice([-1,1],size = (n,n))
    W = np.multiply(sign,np.random.randint(low = 1, high = 5, size = (n,n)))
    return W

def graph_matrix(graph):
    n = graph.number_of_nodes()
    A = nx.adjacency_matrix(graph).todense()
    W = weighted_matrix(n)
    G = np.multiply(W,A)
    spectral_radius = np.max([np.abs(i) for i in np.linalg.eigvals(G)])
    G = G.T/(spectral_radius+0.01)

    return G

## Data Synthesis

In [33]:
def data_synthesis(noise_level, n, l, m, G_truth):
    noise = noise_level * np.random.multivariate_normal(mean = np.zeros(l), cov = np.identity(l), size = m).reshape(l,m)
    tmp = np.identity(n)
    M = np.zeros((n-2, l, m))
    for i in range(n-2):
        tmp = tmp @ G_truth
        M[i] = C@tmp@B + noise
    M = torch.from_numpy(M).float()
    #G_truth = torch.from_numpy(G_truth).float()
    return M

## ADAM

In [34]:
class Model(nn.Module):
    def __init__(self, n, C, B, gamma):
        super().__init__()
        self.n = n
        self.C = torch.from_numpy(C).float()
        self.B = torch.from_numpy(B).float()
        self.G = nn.Parameter(torch.zeros(self.n, self.n), requires_grad=True)
        self.gamma = gamma

    def forward(self):
        # Make G symmetric by averaging it with its transpose
        G_symmetric = 0.5 * (self.G + self.G.transpose(0, 1))
        
        # Set diagonal elements to 0
        G_symmetric_no_diag = G_symmetric * (1 - torch.eye(self.n))
        
        output = torch.zeros((self.n - 2, self.C.shape[0], self.B.shape[1]))
        tmp = torch.eye(self.n)
        l = output.shape[0]
        for i in range(l):
            tmp = tmp @ G_symmetric_no_diag
            output[i] = self.C @ tmp @ self.B
        return output + self.gamma * torch.norm(G_symmetric_no_diag, p=1)
    
def adam(n, C, B, M, gamma):
    model = Model(n, C, B,gamma)
    criterion = nn.MSELoss(reduction='sum')
    #optimizer = optim.Adam(model.parameters(), lr=0.001)

    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    epoch = 20000
    for i in range(epoch):
        output = model.forward()
        output = output.view(output.shape[0], -1)
        M = M.view(M.shape[0], -1)

        loss = criterion(output, M)
        if i % 5000 == 0:
            print("Epoch: [{}/{}]\tLoss: {}".format(i, epoch, loss.detach().numpy()))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    G_learn = model.G.cpu().detach().numpy()
    return G_learn

## Main

In [35]:

n = 10
M = range(0,int(n/2))
O = range(int(n/2) + 1, n)
l, m = len(O), len(M)

B = generate_B(n, m, M)
C = generate_C(n, l, O)

graph_modes = ['er','ws','bara']

p_er = 0.07
k_ws = 1
p_ws = 0.2
m_ba = 1
num_simulations = 3

noise_level = 0

for graph_mode in graph_modes:
    df_graph = []
    df_error = []
    df_method = []
    df_noise = []
    cnt = 1
    while cnt <= num_simulations:
        graph = choose_graph(graph_mode)
        G_truth = graph_matrix(graph)
        obs_rank = np.linalg.matrix_rank(control.obsv(G_truth,C))
        if obs_rank == n:
            print('Processing ', cnt, 'th ',graph_mode,' graph.')
            noise_level = 0
            M = data_synthesis(noise_level, n, l, m, G_truth)
            a2_theta = 0
            G_a2 = adam(n, C, B, M, a2_theta)
            a2_error = np.linalg.norm(G_truth-G_a2,'fro')/np.linalg.norm(G_truth,'fro')
            df_noise.append(noise_level)
            df_graph.append(graph_mode)
            df_error.append(a2_error)
            df_method.append('SSubI')
            print('Relative Error of A2 of ',cnt,'th',graph_mode,' graph: ', a2_error)
            a2_l1_theta = 0.00005
            G_a2_l1 = adam(n, C, B, M, a2_l1_theta)
            a2_l1_error = np.linalg.norm(G_truth-G_a2_l1,'fro')/np.linalg.norm(G_truth,'fro')
            df_noise.append(noise_level)
            df_graph.append(graph_mode)
            df_error.append(a2_l1_error)
            df_method.append('SSSubI')
            print('Relative Error of A2L1 of ',cnt,'th',graph_mode,' graph: ', a2_l1_error)
            cnt = cnt +1
        else:
            pass
    print(len(df_graph),len(df_error),len(df_method),len(df_noise)) 
    columns = {'graph':df_graph,'Noise level':df_noise,'error':df_error,'method':df_method}
    df_performance = pd.DataFrame(columns)
    df_performance.to_csv('./A2_noise_performance_{}.csv'.format(graph_mode))




Processing  1 th  er  graph.
Epoch: [0/20000]	Loss: 17.368236541748047
Epoch: [5000/20000]	Loss: 3.5795674324035645
Epoch: [10000/20000]	Loss: 1.9127371311187744
Epoch: [15000/20000]	Loss: 1.8667042255401611
Relative Error of A2 of  1 th er  graph:  0.8865431567221292
Epoch: [0/20000]	Loss: 17.368236541748047
Epoch: [5000/20000]	Loss: 3.229539632797241
Epoch: [10000/20000]	Loss: 1.8681457042694092
Epoch: [15000/20000]	Loss: 1.8642563819885254
Relative Error of A2L1 of  1 th er  graph:  0.8868721418560239
Processing  2 th  er  graph.
Epoch: [0/20000]	Loss: 5.259052276611328
Epoch: [5000/20000]	Loss: 3.9376420974731445
Epoch: [10000/20000]	Loss: 3.7314367294311523
Epoch: [15000/20000]	Loss: 3.549469470977783
Relative Error of A2 of  2 th er  graph:  1.4042523013004558
Epoch: [0/20000]	Loss: 5.259052276611328
Epoch: [5000/20000]	Loss: 3.8728320598602295
Epoch: [10000/20000]	Loss: 3.5723726749420166
Epoch: [15000/20000]	Loss: 3.305985927581787
Relative Error of A2L1 of  2 th er  graph:  1.

NetworkXError: Graph has no nodes or edges